In [1]:
import os
import pandas as pd
import logging
import re
import ast
import numpy as np 
import torch
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
from sentence_transformers import SentenceTransformer
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderServiceError
from concurrent.futures import ThreadPoolExecutor
from collections import Counter, defaultdict

/home/charles/stkgfs/stkgfs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-26 08:57:55.404931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769417876.121085 3343336 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769417876.331928 3343336 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-26 08:57:58.564991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical 

In [2]:
today_df = pd.read_csv("../results_csv/spatial_entities_with_pos.csv", index_col=False)
today_df

,article_id,segment_id,segment_text,annee,entity_location,entity_location_cam
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
2,1,0,Electrification : Il ny a pas deau pour le cou...,2009,"[{'label': 'loc', 'text': 'la Kompienga', 'sta...","[{'label': 'LOC', 'text': 'Kompienga', 'start'..."
3,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[{'label': 'loc', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
...,...,...,...,...,...,...
3440,998,3,à vie à lencontre de laccusé. 5 jours à compte...,2009,[],[]
3441,998,4,quelle encourait si son fils recouvrait la lib...,2009,"[{'label': 'loc', 'text': 'Ouakara', 'start': ...","[{'label': 'LOC', 'text': 'Ouakara', 'start': ..."
3442,998,5,Par Serge COULIBALY Le Pays,2009,"[{'label': 'loc', 'text': 'Le Pays', 'start': ...","[{'label': 'LOC', 'text': 'Le Pays', 'start': ..."
3443,999,0,AGRESSION DUN HUISSIER DE JUSTICE : Appolinair...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'F CFA', 'start': 75..."


In [4]:
filtered_df_nelson = pd.read_csv("../results_csv/filtered_df_nelson.csv", index_col=False)
filtered_df_nelson

,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,publication_date,heidletime_dates,real_entity_location,real_entity_location_cam,similarity_score,best_term,concept,theme,neutral
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['14 janvier 2009', '14 janvier 2009']","[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...",2009-01-15,"['2009-01-15', '2009-01-14', '2009-01-14', '20...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.672193,crise alimentaire,famine,alimentaire,0
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[""'Ouagadougou'""]",['2007'],"[""'Ouagadougou'""]",['en 2007.'],2009-01-15,['2007'],"{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.726427,inégalité alimentaire,inégalité alimentaire,sociopolitique,0
2,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[""'Bagré'"", ""'Ouagadougou'"", ""'Bagré'"", ""'Komp...",['mi-septembre'],"[""'Bagré'"", ""'CRCO'"", ""'Bobo'"", ""'Ouagadougou'...","['mi-septembre', 'fin de lannée en', 'deux sem...",2009-08-03,"['2009-08-03', '2011', '2009-09']","{'Bagré': {'type': 'department', 'lat': '11.53...","{'Bagré': {'type': 'department', 'lat': '11.53...",0.686193,désapprovisionnement,rupture de stock alimentaire,agriculture,0
3,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'laxe Djib...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[""'Ouagadougou'"", ""'Burkina Faso'"", ""'Ouagadou...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...",2009-06-05,"['2009-07', '2009-08', '2009-09', '2009-06-05'...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.717640,précipitations,pluie,agriculture,1
4,2,1,"les chiffres dans toutes les trois zones, témo...",2009,[''],"['juillet-août-septembre 2009', 'juillet-août-...","[""'GACM'""]","['juillet', 'août-septembre 2009', 'des trente...",2009-06-05,"['2009-07', '2009-08', '2009-09', '2008-07', '...",{},{},0.678547,mauvaise pluviométrie,sécheresse,environment,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,997,1,"putréfaction pour que le corps remonte , comme...",2009,[''],[],"[""'grotte de la Vierge Marie'"", ""'Manga'""]",[],2009-03-16,[],{},"{'Manga': {'type': 'town', 'lat': '11.6673837'...",0.668549,moyen de subsistance en danger,famine,alimentaire,0
1719,998,1,"de ces assises. Parmi elles, 21 crimes de sang...",2009,"[""'Dédougou'"", ""'Bankuy'"", ""'Passakongo'"", ""'B...","['26 avril 2006', 'avril 2006', 'mai 2009']","[""'Dédougou'"", ""'Bankuy'"", ""'Passakongo'"", ""'B...","['le 26 avril 2006.', 'le 7 novembre dernier',...",2009-11-09,"['2006-04-26', '2009-11-07', '2006-04', '2009-...","{'Dédougou': {'type': 'city', 'lat': '12.46291...","{'Dédougou': {'type': 'city', 'lat': '12.46291...",0.529821,violations des droits,violence,sociopolitique,0
1720,998,2,nuit du 25 au 26 avril 2006 dans le but datten...,2009,"[""'Dédougou'"", ""'Tougan -Ouahigouya'"", ""'Touga...",[],"[""'CFA'"", ""'Dédougou'"", ""'Tougan -Ouahigouya'""...","['25', '26 avril 2006 dans le', '5 heures et',...",2009-11-09,"['2006-04-25', '2006-04-26']","{'Dédougou': {'type': 'city', 'lat': '12.46291...","{'Dédougou': {'type': 'city', 'lat': '12.46291...",0.528404,groupe armé,violence,sociopolitique,0
1721,998,5,Par Serge COULIBALY Le Pays,2009,"[""'Le Pays'""]",[],"[""'Le Pays'""]",[],2009-11-09,[],{},{},0.533957,chenille,organismes invasifs et pestes animales,agriculture,0


In [5]:
cols_to_merge = ["article_id", "segment_id", "entity_location", "entity_location_cam"]


merged_df = pd.merge(
    filtered_df_nelson,
    today_df[cols_to_merge],
    on=["article_id", "segment_id"],
    how="left", 
    suffixes=("", "_today") 
)


merged_df["entity_location"] = merged_df["entity_location_today"]
merged_df["entity_location_cam"] = merged_df["entity_location_cam_today"]


merged_df.drop(["entity_location_today", "entity_location_cam_today"], axis=1, inplace=True)
merged_df

,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,publication_date,heidletime_dates,real_entity_location,real_entity_location_cam,similarity_score,best_term,concept,theme,neutral
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","['14 janvier 2009', '14 janvier 2009']","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...",2009-01-15,"['2009-01-15', '2009-01-14', '2009-01-14', '20...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.672193,crise alimentaire,famine,alimentaire,0
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...",['2007'],"[{'label': 'LOC', 'text': 'Ouagadougou', 'star...",['en 2007.'],2009-01-15,['2007'],"{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.726427,inégalité alimentaire,inégalité alimentaire,sociopolitique,0
2,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[{'label': 'loc', 'text': 'Bagré', 'start': 34...",['mi-septembre'],"[{'label': 'LOC', 'text': 'Bagré', 'start': 34...","['mi-septembre', 'fin de lannée en', 'deux sem...",2009-08-03,"['2009-08-03', '2011', '2009-09']","{'Bagré': {'type': 'department', 'lat': '11.53...","{'Bagré': {'type': 'department', 'lat': '11.53...",0.686193,désapprovisionnement,rupture de stock alimentaire,agriculture,0
3,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[{'label': 'loc', 'text': 'Ouagadougou', 'star...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...",2009-06-05,"['2009-07', '2009-08', '2009-09', '2009-06-05'...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...","{'Ouagadougou': {'type': 'city', 'lat': '12.36...",0.717640,précipitations,pluie,agriculture,1
4,2,1,"les chiffres dans toutes les trois zones, témo...",2009,[],"['juillet-août-septembre 2009', 'juillet-août-...","[{'label': 'LOC', 'text': 'Centre africain', '...","['juillet', 'août-septembre 2009', 'des trente...",2009-06-05,"['2009-07', '2009-08', '2009-09', '2008-07', '...",{},{},0.678547,mauvaise pluviométrie,sécheresse,environment,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,997,1,"putréfaction pour que le corps remonte , comme...",2009,[],[],"[{'label': 'LOC', 'text': 'grotte de la Vierge...",[],2009-03-16,[],{},"{'Manga': {'type': 'town', 'lat': '11.6673837'...",0.668549,moyen de subsistance en danger,famine,alimentaire,0
1719,998,1,"de ces assises. Parmi elles, 21 crimes de sang...",2009,"[{'label': 'loc', 'text': 'Dédougou', 'start':...","['26 avril 2006', 'avril 2006', 'mai 2009']","[{'label': 'LOC', 'text': 'Dédougou', 'start':...","['le 26 avril 2006.', 'le 7 novembre dernier',...",2009-11-09,"['2006-04-26', '2009-11-07', '2006-04', '2009-...","{'Dédougou': {'type': 'city', 'lat': '12.46291...","{'Dédougou': {'type': 'city', 'lat': '12.46291...",0.529821,violations des droits,violence,sociopolitique,0
1720,998,2,nuit du 25 au 26 avril 2006 dans le but datten...,2009,"[{'label': 'loc', 'text': 'Dédougou', 'start':...",[],"[{'label': 'LOC', 'text': 'F CFA', 'start': 40...","['25', '26 avril 2006 dans le', '5 heures et',...",2009-11-09,"['2006-04-25', '2006-04-26']","{'Dédougou': {'type': 'city', 'lat': '12.46291...","{'Dédougou': {'type': 'city', 'lat': '12.46291...",0.528404,groupe armé,violence,sociopolitique,0
1721,998,5,Par Serge COULIBALY Le Pays,2009,"[{'label': 'loc', 'text': 'Le Pays', 'start': ...",[],"[{'label': 'LOC', 'text': 'Le Pays', 'start': ...",[],2009-11-09,[],{},{},0.533957,chenille,organismes invasifs et pestes animales,agriculture,0


In [7]:
old_df = pd.read_csv("../results_csv/entites_df.csv", index_col=False)
old_df

,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,filtered_entity_location,filtered_entity_location_cam,real_entity_location,real_entity_location_cam
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['14 janvier 2009', '14 janvier 2009']","[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","{""'Ouagadougou'"": {'type': 'city', 'count': 6}}","{""'Ouagadougou'"": {'type': 'city', 'count': 6}..."
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[""'Ouagadougou'""]",['2007'],"[""'Ouagadougou'""]",['en 2007.'],['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","{""'Ouagadougou'"": {'type': 'city', 'count': 1}}"
2,1,0,Electrification : Il ny a pas deau pour le cou...,2009,"[""'la Kompienga'"", ""'Bagré'"", ""'Ouagadougou'"",...","['lundi 3 août 2009', 'vendredi 31 juillet 200...","[""'la Kompienga'"", ""'Bagré'"", ""'Centre régiona...","['ndi 3 août 2009 à 02h25min', 'le vendredi 31...","['Boulgou', 'Kadiogo', 'Boulgou']","['Boulgou', 'Kompienga', 'Boulgou']","{""'la Kompienga'"": {'type': 'department', 'cou...","{""'la Kompienga'"": {'type': 'department', 'cou..."
3,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[""'Bagré'"", ""'Ouagadougou'"", ""'Bagré'"", ""'Komp...",['mi-septembre'],"[""'Bagré'"", ""'CRCO'"", ""'Bobo'"", ""'Ouagadougou'...","['mi-septembre', 'fin de lannée en', 'deux sem...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","{""'Bagré'"": {'type': 'department', 'count': 2}...","{""'Bagré'"": {'type': 'department', 'count': 2}..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'laxe Djib...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[""'Ouagadougou'"", ""'Burkina Faso'"", ""'Ouagadou...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...","['Kadiogo', 'Kadiogo']","['Kadiogo', 'Kadiogo']","{""'Ouagadougou'"": {'type': 'city', 'count': 2}}","{""'Ouagadougou'"": {'type': 'city', 'count': 2}..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3440,998,3,à vie à lencontre de laccusé. 5 jours à compte...,2009,[''],['26 janvier 2005'],[''],"['5 jours à', 'à 38 ans', 'le 26 janvier 2005....",[],[],{},{}
3441,998,4,quelle encourait si son fils recouvrait la lib...,2009,"[""'Ouakara'"", ""'Dédougou'""]",['3 août 2003'],"[""'Ouakara'"", ""'Dédougou'"", ""'Pays'""]","['a 5 jours', 'le 3 août 2003 à']","['Mouhoun', 'Mouhoun']","['Mouhoun', 'Mouhoun']","{""'Dédougou'"": {'type': 'city', 'count': 1}}","{""'Dédougou'"": {'type': 'city', 'count': 1}}"
3442,998,5,Par Serge COULIBALY Le Pays,2009,"[""'Le Pays'""]",[],"[""'Le Pays'""]",[],[],[],{},{}
3443,999,0,AGRESSION DUN HUISSIER DE JUSTICE : Appolinair...,2009,"[""'Ouagadougou'""]","['mardi 17 mars 2009', '16 mars 2009']","[""'Ouagadougou'""]","['mardi 17 mars 2009 à 00h17min', 'le 16 mars ...",['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","{""'Ouagadougou'"": {'type': 'city', 'count': 1}}"


In [10]:
cols_to_merge = ["article_id", "segment_id", "entity_location", "entity_location_cam"]


merged_df_original = pd.merge(
    old_df,
    today_df[cols_to_merge],
    on=["article_id", "segment_id"],
    how="left", 
    suffixes=("", "_today")
)

merged_df_original

,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,filtered_entity_location,filtered_entity_location_cam,real_entity_location,real_entity_location_cam,entity_location_today,entity_location_cam_today
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['14 janvier 2009', '14 janvier 2009']","[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","{""'Ouagadougou'"": {'type': 'city', 'count': 6}}","{""'Ouagadougou'"": {'type': 'city', 'count': 6}...","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[""'Ouagadougou'""]",['2007'],"[""'Ouagadougou'""]",['en 2007.'],['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
2,1,0,Electrification : Il ny a pas deau pour le cou...,2009,"[""'la Kompienga'"", ""'Bagré'"", ""'Ouagadougou'"",...","['lundi 3 août 2009', 'vendredi 31 juillet 200...","[""'la Kompienga'"", ""'Bagré'"", ""'Centre régiona...","['ndi 3 août 2009 à 02h25min', 'le vendredi 31...","['Boulgou', 'Kadiogo', 'Boulgou']","['Boulgou', 'Kompienga', 'Boulgou']","{""'la Kompienga'"": {'type': 'department', 'cou...","{""'la Kompienga'"": {'type': 'department', 'cou...","[{'label': 'loc', 'text': 'la Kompienga', 'sta...","[{'label': 'LOC', 'text': 'Kompienga', 'start'..."
3,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[""'Bagré'"", ""'Ouagadougou'"", ""'Bagré'"", ""'Komp...",['mi-septembre'],"[""'Bagré'"", ""'CRCO'"", ""'Bobo'"", ""'Ouagadougou'...","['mi-septembre', 'fin de lannée en', 'deux sem...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","{""'Bagré'"": {'type': 'department', 'count': 2}...","{""'Bagré'"": {'type': 'department', 'count': 2}...","[{'label': 'loc', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'laxe Djib...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[""'Ouagadougou'"", ""'Burkina Faso'"", ""'Ouagadou...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...","['Kadiogo', 'Kadiogo']","['Kadiogo', 'Kadiogo']","{""'Ouagadougou'"": {'type': 'city', 'count': 2}}","{""'Ouagadougou'"": {'type': 'city', 'count': 2}...","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3440,998,3,à vie à lencontre de laccusé. 5 jours à compte...,2009,[''],['26 janvier 2005'],[''],"['5 jours à', 'à 38 ans', 'le 26 janvier 2005....",[],[],{},{},[],[]
3441,998,4,quelle encourait si son fils recouvrait la lib...,2009,"[""'Ouakara'"", ""'Dédougou'""]",['3 août 2003'],"[""'Ouakara'"", ""'Dédougou'"", ""'Pays'""]","['a 5 jours', 'le 3 août 2003 à']","['Mouhoun', 'Mouhoun']","['Mouhoun', 'Mouhoun']","{""'Dédougou'"": {'type': 'city', 'count': 1}}","{""'Dédougou'"": {'type': 'city', 'count': 1}}","[{'label': 'loc', 'text': 'Ouakara', 'start': ...","[{'label': 'LOC', 'text': 'Ouakara', 'start': ..."
3442,998,5,Par Serge COULIBALY Le Pays,2009,"[""'Le Pays'""]",[],"[""'Le Pays'""]",[],[],[],{},{},"[{'label': 'loc', 'text': 'Le Pays', 'start': ...","[{'label': 'LOC', 'text': 'Le Pays', 'start': ..."
3443,999,0,AGRESSION DUN HUISSIER DE JUSTICE : Appolinair...,2009,"[""'Ouagadougou'""]","['mardi 17 mars 2009', '16 mars 2009']","[""'Ouagadougou'""]","['mardi 17 mars 2009 à 00h17min', 'le 16 mars ...",['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type':

In [11]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderServiceError
from collections import Counter
import pandas as pd
import re

# Initialisation du géocodeur
geolocator = Nominatim(user_agent="myencoder", timeout=10)

# Types d'entités spatiales valides
VALID_TYPES = {"city", "department", "province", "country", "town", "region"}

# Fonction pour géocoder une entité et vérifier si elle est valide
def geocode_and_filter(entity):
    try:
        location = geolocator.geocode(entity, addressdetails=True)
        if location and location.raw.get('addresstype') in VALID_TYPES:
            return True
    except GeocoderServiceError as e:
        print(f"Erreur de géocodage pour '{entity}': {e}")
    return False

# Fonction pour filtrer les entités
def filter_entities(entities):
    return [entity for entity in entities if geocode_and_filter(entity)]


strict_case_entities = {"Est", "Nord", "Centre"}

def find_gold_annotations(text, entities):
    entities_sorted = sorted(entities, key=len, reverse=True)
    final_matches = []

    for entity in entities_sorted:
        pattern = r"\b" + re.escape(entity) + r"\b"
        if entity in strict_case_entities:
            matches = re.finditer(pattern, text)
        else:
            matches = re.finditer(pattern, text, flags=re.IGNORECASE)

        for match in matches:
            start, end = match.start()-1, match.end()
            overlap = any(not (end <= fm['start'] or start >= fm['end']) for fm in final_matches)

            if not overlap:
                matched_text = text[start:end]
                final_matches.append({
                    'label': 'LOC',
                    'text': matched_text,
                    'start': start,
                    'end': end
                })

    final_matches.sort(key=lambda m: m['start'])
    return final_matches

In [15]:
import requests
from bs4 import BeautifulSoup
import re

def extraire_communes_burkina():
    url = "https://fr.wikipedia.org/wiki/Liste_de_villes_du_Burkina_Faso"
    
    # Ajouter un User-Agent pour éviter le blocage
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Chercher le tableau avec une classe contenant 'wikitable'
    tableau = soup.find('table', class_=lambda x: x and 'wikitable' in x)
    
    # Vérification que le tableau existe
    if tableau is None:
        print("Aucun tableau trouvé sur la page")
        # Debug: afficher les tables disponibles
        tables = soup.find_all('table')
        print(f"Tables trouvées: {len(tables)}")
        for i, t in enumerate(tables):
            print(f"  Table {i}: class={t.get('class')}")
        return []
    
    communes = set()
    
    for ligne in tableau.find_all('tr')[1:]:
        colonnes = ligne.find_all('td')
        if colonnes:
            nom = colonnes[0].get_text().strip()
            matches = re.findall(r'([^()]+)(?:\s*\((?:ou\s+)?([^)]+)\))?', nom)
            for match in matches:
                if match[0].strip():
                    communes.add(match[0].strip())
                if len(match) > 1 and match[1] and match[1].strip():
                    communes.add(match[1].strip())
    
    return sorted(list(communes))

communes = extraire_communes_burkina()
print(f"Nombre total de communes trouvées : {len(communes)}")

Nombre total de communes trouvées : 379


In [16]:
gold_spatial_entities = communes[7:370]
len(gold_spatial_entities)

363

In [17]:
gold_spatial_entities = ['Boucle du Mouhoun',
    'Cascades',
    'Centre',
    'Centre-Est',
    'Centre-Nord',
    'Centre-Ouest',
    'Centre-Sud',
    'Est',
    'Hauts-Bassins',
    'Nord',
    'Plateau-Central',
    'Sahel',
    'Sud-Ouest',
  'Absouya',
  'Bobo',
  'Burkina',
  'Burkina Faso',
  'Burkina-Faso',
  'Ouaga',
  'Haute Volta',
  'Haute-Volta',
 'Andemtenga',
 'Arbinda',
 'Arbollé',
 'Arbolé',
 'Bagaré',
 'Bagassi',
 'Bagré',
 'Bakata',
 'Balavé',
 'Bama',
 'Bana',
 'Banfora',
 'Banh',
 'Bani',
 'Banzon',
 'Bané',
 'Baraboulé',
 'Barani',
 'Barga',
 'Barsalogho',
 'Barthiébougou, Bartibougou',
 'Bartiébougou',
 'Baskouré',
 'Bassi',
 'Batié',
 'Bilanga',
 'Bindé',
 'Bingo',
 'Bissiga',
 'Bitou',
 'Bittou',
 'Biéha',
 'Boala',
 'Bobo-Dioulasso',
 'Bogandé',
 'Bokin',
 'Bomborokuy',
 'Bondigui',
 'Bondokuy',
 'Boni',
 'Bony',
 'Boromo',
 'Botou',
 'Boudry',
 'Bougnounou',
 'Boulsa',
 'Boundoré',
 'Boura',
 'Bourasso',
 'Bouroum',
 'Bouroum-Bouroum',
 'Bourzanga',
 'Boussou',
 'Boussoukoula',
 'Boussouma',
 'Boussé',
 'Bousséra',
 'Béguédo',
 'Békuy',
 'Béré',
 'Béréba',
 'Bérégadougou',
 'Cassou',
 'Coalla',
 'Comin-Yanga',
 'Dablo',
 'Dakoro',
 'Dalo',
 'Dandé',
 'Dano',
 'Dapélogo',
 'Dargo',
 'Dassa',
 'Di',
 'Diabo',
 'Dialgaye',
 'Diapaga',
 'Diapangou',
 'Didyr',
 'Diguel',
 'Dissin',
 'Diébougou',
 'Djibasso',
 'Djibo',
 'Djigoué',
 'Djigouéra',
 'Dokuy',
 'Dolo',
 'Dori',
 'Doulougou',
 'Doumbala',
 'Douna',
 'Douroula',
 'Dourtenga',
 'Dédougou',
 'Déou',
 'Fada N’Gourma',
 'Falagountou',
 'Fara',
 'Faramana',
 'Founzan',
 'Foutouri',
 'Fô',
 'Gao',
 'Gaongo',
 'Gaoua',
 'Garango',
 'Gassan',
 'Gayéri',
 'Gbomblora',
 'Godyr',
 'Gogo',
 'Gomboro',
 'Gomboussougou',
 'Gomponsom',
 'Gorgadji',
 'Gorom-Gorom',
 'Gossina',
 'Gounghin',
 'Gourcy',
 'Guiaro',
 'Guiba',
 'Guibaré',
 'Guéguéré',
 'Houndé',
 'Imasgho',
 'Imasgo',
 'Iolonioro',
 'Ipelcé',
 'Kalsaka',
 'Kampti',
 'Kando',
 'Kangala',
 'Kankalaba',
 'Kantchari',
 'Karangasso-Sambla',
 'Karangasso-Vigué',
 'Karankasso-Sambla',
 'Karankasso-Vigué',
 'Kassoum',
 'Kaya',
 'Kayan',
 'Kayao',
 'Kaïn',
 'Kelbo',
 'Khyon',
 'Kiembara',
 'Kindi',
 'Kirsi',
 'Koalla',
 'Kogho',
 'Kokologho',
 'Koloko',
 'Kombissiri',
 'Kombori',
 'Komki-Ipala',
 'Kompienga',
 'Komsilga',
 'Komtoèga',
 'Kona',
 'Kongoussi',
 'Koper',
 'Kordié',
 'Korsimoro',
 'Kossouka',
 'Koti',
 'Koubri',
 'Koudougou',
 'Kougny',
 'Kouka',
 'Koumbia',
 'Koumbri',
 'Koundougou',
 'Koupéla',
 'Kourignon',
 'Kourinion',
 'Kourouma',
 'Koutougou',
 'Kpuéré',
 'Kyon',
 'Lalgaye',
 'Lanfièra',
 'Lankoué',
 'Laye',
 'Legmoin',
 'Liptougou',
 'Logobou',
 'Loropéni',
 'Loumana',
 'Loumbila',
 'Lâ-Todin',
 'Léba',
 'Léna',
 'Léo',
 'Madjoari',
 'Madouba',
 'Malba',
 'Manga',
 'Mangodara',
 'Mani',
 'Manni',
 'Mansila',
 'Mané',
 'Markoye',
 'Matiacoali',
 'Midebdo',
 'Mogtédo',
 'Morolaba',
 'Moussodougou',
 'Méguet',
 'Nagbingou',
 'Nagréongo',
 'Nako',
 'Namissiguima',
 'Namounou',
 'Nandiala',
 'Nanoro',
 'Nassoumbou',
 'Nasséré',
 'Niabouri',
 'Niangoloko',
 'Niankorodougou',
 'Niaogho',
 'Niaogo',
 'Niou',
 'Niégo',
 'Nobéré',
 'Nouna',
 'Nébiélianayou',
 'N’Dorola',
 'Orodara',
 'Oronkua',
 'Ouagadougou',
 'Ouahigouya',
 'Ouargaye',
 'Ouarkoye',
 'Ouessa',
 'Ouindigui',
 'Oula',
 'Ouo',
 'Ourgou-Manéga',
 'Oursi',
 'Oury',
 'Ouéléni',
 'Pabré',
 'Padéma',
 'Pama',
 'Partiaga',
 'Pella',
 'Pensa',
 'Pibaoré',
 'Pilimpikou',
 'Pissila',
 'Piéla',
 'Poa',
 'Pobé-Mengao',
 'Pompoï',
 'Pouni',
 'Poura',
 'Pouytenga',
 'Pâ',
 'Péni',
 'Périgban',
 'Pérignan',
 'Pô',
 'Rambo',
 'Ramongo',
 'Rollo',
 'Rouko',
 'Réo',
 'Saaba',
 'Sabcé',
 'Sabou',
 'Safané',
 'Salogo',
 'Samba',
 'Sami',
 'Samogohiri',
 'Samorogouan',
 'Sampelga',
 'Sanaba',
 'Sanga',
 'Sangha',
 'Saponé',
 'Sapouy',
 'Satiri',
 'Sebba',
 'Seytenga',
 'Siby',
 'Sidéradougou',
 'Siglé',
 'Silly',
 'Sindo',
 'Sindou',
 'Soaw',
 'Solenzo',
 'Solhan',
 'Sollé',
 'Sono',
 'Soubakaniédougou',
 'Soudougui',
 'Sourgou',
 'Sourgoubila',
 'Séguénéga',
 'Tambaga',
 'Tangaye',
 'Tanghin-Dassouri',
 'Tankougounadié',
 'Tansarga',
 'Tansila',
 'Tchériba',
 'Tenkodogo',
 'Tensobentenga',
 'Thion',
 'Thiou',
 'Thyou',
 'Tiankoura',
 'Tibga',
 'Tikaré',
 'Tin-Akoff',
 'Titabé',
 'Titao',
 'Tiébélé',
 'Tiéfora',
 'Toma',
 'Tongomayel',
 'Tougan',
 'Tougo',
 'Tougouri',
 'Toussiana',
 'Toécé',
 'Toéghin',
 'Toéni',
 'Ténado',
 'Tô',
 'Wolonkoto',
 'Yaba',
 'Yaho',
 'Yako',
 'Yalgo',
 'Yamba',
 'Yargatenga',
 'Yargo',
 'Yondé',
 'Yé',
 'Zabré',
 'Zam',
 'Zambo',
 'Zamo',
 'Zawara',
 'Zecco',
 'Ziga',
 'Zimtenga',
 'Ziniaré',
 'Ziou',
 'Zitenga',
 'Zoaga',
 'Zogoré',
 'Zonsé',
 'Zorgho',
 'Zoungou',
 'Zéguédéguin']

In [18]:
len(gold_spatial_entities)

383

In [19]:
filtered_entities = filter_entities(gold_spatial_entities)

merged_df_original['gold_annotations'] = merged_df_original['segment_text'].apply(
    lambda txt: find_gold_annotations(txt, filtered_entities)
)


In [20]:
len(filtered_entities)

272

In [21]:
import ast

merged_df_original['real_entity_cam_location_cam'] = merged_df_original['real_entity_location_cam'].apply(ast.literal_eval)

def safe_eval_list(x):
    if pd.isna(x) or x in ["", "nan"]:
        return []
    try:
        val = ast.literal_eval(x)
        # val devrait alors être une liste de dicts si le format est correct
        if isinstance(val, list):
            return val
        else:
            return []
    except:
        return []

merged_df_original['entity_location_cam_today'] = merged_df_original['entity_location_cam_today'].apply(safe_eval_list)

In [22]:
import ast
import json

def build_burkina_pred(row):
    try:
        # Convertir la chaîne JSON en dictionnaire pour real_entity_location
        if isinstance(row['real_entity_location_cam'], str):
            real_loc_dict = ast.literal_eval(row['real_entity_location_cam'])
        else:
            real_loc_dict = row['real_entity_location_cam']
            
        if not real_loc_dict:
            return []
            
        valid_locations = {k.strip("'").strip() for k in real_loc_dict.keys()}
        
        if isinstance(row['entity_location_cam_today'], str):
            predictions = ast.literal_eval(row['entity_location_cam_today'])
        else:
            predictions = row['entity_location_cam_today']
            
        # Filtrer les prédictions
        filtered_predictions = []
        for prediction in predictions:
            if prediction['text'].strip() in valid_locations:
                filtered_predictions.append({
                    'label': prediction['label'],
                    'text': prediction['text'],
                    'start': prediction['start'],
                    'end': prediction['end']
                })
                
        return filtered_predictions
        
    except Exception as e:
        print(f"Erreur lors du traitement : {e}")
        return []
    
merged_df_original['burkina_pred'] = merged_df_original.apply(build_burkina_pred, axis=1)

In [23]:
merged_df_original

,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,filtered_entity_location,filtered_entity_location_cam,real_entity_location,real_entity_location_cam,entity_location_today,entity_location_cam_today,gold_annotations,real_entity_cam_location_cam,burkina_pred
0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['14 janvier 2009', '14 janvier 2009']","[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","{""'Ouagadougou'"": {'type': 'city', 'count': 6}}","{""'Ouagadougou'"": {'type': 'city', 'count': 6}...","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","{''Ouagadougou'': {'type': 'city', 'count': 6}...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
1,0,1,ont montré que la diversité alimentaire était ...,2009,"[""'Ouagadougou'""]",['2007'],"[""'Ouagadougou'""]",['en 2007.'],['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","{''Ouagadougou'': {'type': 'city', 'count': 1}}","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
2,1,0,Electrification : Il ny a pas deau pour le cou...,2009,"[""'la Kompienga'"", ""'Bagré'"", ""'Ouagadougou'"",...","['lundi 3 août 2009', 'vendredi 31 juillet 200...","[""'la Kompienga'"", ""'Bagré'"", ""'Centre régiona...","['ndi 3 août 2009 à 02h25min', 'le vendredi 31...","['Boulgou', 'Kadiogo', 'Boulgou']","['Boulgou', 'Kompienga', 'Boulgou']","{""'la Kompienga'"": {'type': 'department', 'cou...","{""'la Kompienga'"": {'type': 'department', 'cou...","[{'label': 'loc', 'text': 'la Kompienga', 'sta...","[{'label': 'LOC', 'text': 'Kompienga', 'start'...","[{'label': 'LOC', 'text': ' Kompienga', 'start...","{''la Kompienga'': {'type': 'department', 'cou...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
3,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[""'Bagré'"", ""'Ouagadougou'"", ""'Bagré'"", ""'Komp...",['mi-septembre'],"[""'Bagré'"", ""'CRCO'"", ""'Bobo'"", ""'Ouagadougou'...","['mi-septembre', 'fin de lannée en', 'deux sem...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","{""'Bagré'"": {'type': 'department', 'count': 2}...","{""'Bagré'"": {'type': 'department', 'count': 2}...","[{'label': 'loc', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': ' Bagré', 'start': 3...","{''Bagré'': {'type': 'department', 'count': 2}...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'laxe Djib...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[""'Ouagadougou'"", ""'Burkina Faso'"", ""'Ouagadou...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...","['Kadiogo', 'Kadiogo']","['Kadiogo', 'Kadiogo']","{""'Ouagadougou'"": {'type': 'city', 'count': 2}}","{""'Ouagadougou'"": {'type': 'city', 'count': 2}...","[{'label': 'loc', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","{''Ouagadougou'': {'type': 'city', 'count': 2}...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3440,998,3,à vie à lencontre de laccusé. 5 jours à compte...,2009,[''],['26 janvier 2005'],[''],"['5 jours à', 'à 38 ans', 'le 26 janvier 2005....",[],[],{},{},[],[],[],{},[]
3441,998,4,quelle encourait si son fils recouvrait

In [24]:
metrics_merged_df = merged_df_original[['article_id', 'segment_id', 'segment_text', 'gold_annotations', 'burkina_pred']]
metrics_merged_df

,article_id,segment_id,segment_text,gold_annotations,burkina_pred
0,0,0,Insécurité alimentaire en milieu urbain : La v...,"[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
1,0,1,ont montré que la diversité alimentaire était ...,"[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
2,1,0,Electrification : Il ny a pas deau pour le cou...,"[{'label': 'LOC', 'text': ' Kompienga', 'start...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
3,1,1,"actuellement, ne sont pas au mieux de leur tau...","[{'label': 'LOC', 'text': ' Bagré', 'start': 3...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
4,2,0,Prévision pluviométrique des mois de juillet-a...,"[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
...,...,...,...,...,...
3440,998,3,à vie à lencontre de laccusé. 5 jours à compte...,[],[]
3441,998,4,quelle encourait si son fils recouvrait la lib...,"[{'label': 'LOC', 'text': ' Dédougou', 'start'...","[{'label': 'LOC', 'text': 'Dédougou', 'start':..."
3442,998,5,Par Serge COULIBALY Le Pays,[],[]
3443,999,0,AGRESSION DUN HUISSIER DE JUSTICE : Appolinair...,"[{'label': 'LOC', 'text': ' Ouagadougou', 'sta...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."


In [25]:
def extract_labels(annotations, text_length):
    labels = ['O'] * text_length
    for annotation in annotations:
        start = annotation['start']
        end = annotation['end']
        label = annotation['label']
        labels[start] = f'B-{label}'
        for i in range(start + 1, end):
            labels[i] = f'I-{label}'
    return labels

y_true = []
y_pred = []

for index, row in metrics_merged_df.iterrows():
    gold_annotations = row['gold_annotations']
    burkina_pred = row['burkina_pred']
    segment_text = row['segment_text']

    true_labels = extract_labels(gold_annotations, len(segment_text))
    pred_labels = extract_labels(burkina_pred, len(segment_text))

    y_true.append(true_labels)
    y_pred.append(pred_labels)


In [26]:
from seqeval.metrics import classification_report, f1_score

print(classification_report(y_true, y_pred))
print(f'F1 Score: {f1_score(y_true, y_pred)}')

              precision    recall  f1-score   support

         LOC       0.82      0.78      0.80      5512

   micro avg       0.82      0.78      0.80      5512
   macro avg       0.82      0.78      0.80      5512
weighted avg       0.82      0.78      0.80      5512

F1 Score: 0.7978436657681941
